In [23]:
import abc
import re
class Analyzer(object):
    __metaclass__  = abc.ABCMeta
    
    @abc.abstractmethod
    def tokenize(self, sentence):
        pass

In [192]:
class SpaceTokenAnalyzer(Analyzer):
    def __init__(self, stopWords=[]):
        self.stopWords = stopWords
        
    def tokenize(self, sentence):
        if not isinstance(sentence, str):
            raise Value("Need string parameter.")  
        tokens = sentence.split()
        return [ t.strip() for t in tokens if t.strip() not in self.stopWords ]

In [228]:
import numpy as np
import pandas as pd

class Vocab(object):
    
    def __init__(self, sentences, analyzer=SpaceTokenAnalyzer()):
        if not isinstance(analyzer, Analyzer):
            raise ValueError("Need Analyzer type.")
        self.analyzer = analyzer
        
        if not isinstance(sentences, list):
            raise VaalueError("Need list parameter.")
        vocab_dict = {}
        for i, sentence in enumerate(sentences):
            vocab_dict[sentence] = dict((token, 1) for token in self.analyzer.tokenize(sentence))
        self.table = pd.DataFrame.from_records(vocab_dict).fillna(0).astype(int).T
        self.table.insert(0, "UNK", 0)
        self.table.insert(0, "EOS", 0)
        self.table.insert(0, "BOS", 0)
        self.table.insert(0, "PAD", 0)
    
    def size(self):
        return len(self.table.columns)
    
    def matrix(self):
        """
        获取句子向量表
        """
        return self.table.as_matrix()
    
    def __vector(self, sentence):
        if sentence in self.table.index:
            return self.table.loc[sentence,]
        else:
            record = {sentence: dict((token, 1) for token in self.analyzer.tokenize(sentence))}
            self.table = self.table.append(pd.DataFrame.from_records(record).T).fillna(0).astype(int)
            return self.table.loc[sentence,]

        
    def vector(self, sentence):
        """
        返回传入句子对应的向量
        """
        if isinstance(sentence, str):
            return self.__vector(sentence)
        
        if isinstance(sentence, list):
            return np.array([self.__vector(s) for s in sentence])
    
    def __sentence(self, vector):
        return self.table.columns[vector].values
    
    def sentence(self, vector):
        """
        返回给定向量对应的单词
        """
        if not isinstance(vector, np.ndarray):
            raise ValueError("Need numpy.ndarray parameter.")
        if len(vector.shape) == 1:
            return self.__sentence(vector)
        
        if len(vector.shape) == 2:
            return np.array([self.__sentence(v) for v in vector])
        raise ValueError("vector's shape must be 1 or 2.")

    

In [229]:
count = 0;
sentences = []
with open("data/cikm_english_train_20180516.txt", 'r') as f:
    for line in f:
        count = count + 1
        if count == 10:
            break
        s = line.split("\t")
        sentences.append(s[1])
        sentences.append(s[3])        
    print sentences

['hola, hago clic en el producto recibido', 'Compr\xc3\xa9 un producto y no he recibido un correo electr\xc3\xb3nico de confirmaci\xc3\xb3n', '\xc2\xa1Hola! Cerr\xc3\xa9 la disputa el 21 de mayo de 2017 y dice que se realizar\xc3\xa1 el reembolso. \xc2\xbfCu\xc3\xa1ndo se reembolsar\xc3\xa1 el dinero?', 'No obtuve el reembolso de mi dinero. Han pasado dos meses. \xc2\xbfCu\xc3\xa1ndo devolver\xc3\xa1 dinero?', 'Orden\xc3\xa9 de Espa\xc3\xb1a a Espa\xc3\xb1a ahora que mandan el pedido de China?', 'Mi pedido lleg\xc3\xb3 pero el color es diferente al que ped\xc3\xad.', '\xc2\xbfDebo pagar impuestos personalizados?', 'C\xc3\xb3mo pagar los derechos de aduana', 'No recib\xc3\xad mi pedido?', 'Mi pedido muestra que no he pagado, pero lo hice', 'Hola Eva Al principio la pregunta. \xc2\xbfPuedo escribir en alem\xc3\xa1n?', '\xc2\xbfC\xc3\xb3mo puedo escribir comentarios para los productos?', 'estoy tan enojado', 'Abr\xc3\xad una disputa y hasta ahora nada', 'No puedo abrir una disputa', 'No p

In [230]:
vocab = Vocab(sentences, analyzer=SpaceTokenAnalyzer(stopWords=[]))
#print vocab.table
#print vocab.table.append(pd.DataFrame.from_records(r).T)
#print vocab.table.as_matrix()
#print vocab.table.columns[np.array([0,2])].values
#vocab.sentence(np.array([0,2]))
#print vocab.vector("hi ma zhi yong")
#print vocab.vector("hello world")
#vocab.table.loc["hello world":"hello wrold",]
#print vocab.matrix()

In [231]:
print vocab.size()

102


In [232]:
print vocab.vector("hola, hago clic en el producto recibido")[0:,]

PAD                0
BOS                0
EOS                0
UNK                0
2017               0
21                 0
Abrí               0
Al                 0
Cerré              0
China?             0
Compré             0
Cómo               0
España             0
Eva                0
Han                0
Hola               0
Mi                 0
No                 0
Ordené             0
a                  0
abrir              0
aduana             0
ahora              0
al                 0
alemán?            0
clic               1
color              0
comentarios        0
confirmación       0
correo             0
                  ..
pedí.              0
pero               0
personalizados?    0
pregunta.          0
principio          0
producto           1
productos?         0
puedo              0
que                0
rastreador         0
rastrear.          0
realizará          0
recibido           1
recibí             0
reembolsará        0
reembolso          0
reembolso.   